# 第十章 评估（下）——当不存在一个简单的正确答案时

 - [一、环境配置](#一、环境配置)
 - [二、运行问答系统获得一个复杂回答](#二、运行问答系统获得一个复杂回答)
 - [三、使用 GPT 评估回答是否正确](#三、使用-GPT-评估回答是否正确)
 - [四、给出一个标准回答，要求其评估生成回答与标准回答的差距](#四、给出一个标准回答，要求其评估生成回答与标准回答的差距)


在上一章中，了解了如何评估 LLM 模型在 **有明确正确答案** 的情况下的输出，我们可以编写一个函数来判断 LLM 输出是否正确地分类并列出产品。

然而，如果 LLM 用于生成文本，而不仅仅是分类问题的答案呢？接下来，我们将探讨如何评估这种类型的 LLM 输出的方法。

## 一、环境配置

同上一章，我们首先需要配置使用 OpenAI API 的环境

In [4]:
import sys
sys.path.append("../")

import tools
from message_processor import process_message
from src.utils.chat_robot import OpenAiChat

## 二、运行问答系统获得一个复杂回答

In [2]:
'''
注意：限于模型对中文理解能力较弱，中文 Prompt 可能会随机出现不成功，可以多次运行；也非常欢迎同学探究更稳定的中文 Prompt
'''
# 用户消息
customer_msg = f"""
告诉我有关 the smartx pro phone 和 the fotosnap camera, the dslr one 的信息。
另外，你们这有什么 TVs ？"""

# 从问题中抽取商品名
_response, _ = process_message(customer_msg, [])
_response

第二步：抽取出商品列表
  ==> [{"category":"智能手机和配件","products":["SmartXProPhone"]},{"category":"相机和摄像机","products":["FotoSnapDSLRCamera"]},{"category":"电视和家庭影院系统","products":["CineView4KTV","CineView8KTV","CineViewOLEDTV"]}]
第三步：查找抽取出的商品信息
  ==> {"name":"SmartXProPhone","category":"智能手机和配件","brand":"SmartX","model_number":"SX-PP10","warranty":"1year","rating":4.6,"features":["6.1-inchdisplay","128GBstorage","12MPdualcamera","5G"],"description
第四步：生成用户回答
  ==> 关于产品信息：1.**SmartXProPhone**:-品牌：SmartX-型号：SX-PP10-价格：899.99美元-主要特点：6.1英寸显示屏、128GB存储、12MP双摄像头、5G支持-描述：一款拥有先进摄像功能的强大智能手机。-保修：1年-评分：4.62.**FotoSnapDSLRCamera**:-品牌：FotoSnap-型号：FS-DSLR200-价格：599.99美元-主要特点：24.2MP传感器、1080p视频、3英寸LCD、可更换镜头-描述：使用这款多功能的单反相机，捕捉惊艳的照片和视频。-保修：1年-评分：4.7关于电视：我们有以下电视可供选择：1.**CineView4KTV**:-品牌：CineView-型号：CV-4K55-价格：599.99美元-主要特点：55英寸显示屏、4K分辨率、HDR、智能电视-描述：一款色彩鲜艳、智能功能丰富的惊艳4K电视。-保修：2年-评分：4.82.**CineView8KTV**:-品牌：CineView-型号：CV-8K65-价格：2999.99美元-主要特点：65英寸显示屏、8K分辨率、HDR、智能电视-描述：通过这款惊艳的8K电视，体验未来。
第六步：模型评估该回答
  ==> {"flag":1,"reas

'关于产品信息：\n\n1. **SmartX ProPhone**:\n   - 品牌：SmartX\n   - 型号：SX-PP10\n   - 价格：899.99美元\n   - 主要特点：6.1英寸显示屏、128GB存储、12MP双摄像头、5G支持\n   - 描述：一款拥有先进摄像功能的强大智能手机。\n   - 保修：1年\n   - 评分：4.6\n\n2. **FotoSnap DSLR Camera**:\n   - 品牌：FotoSnap\n   - 型号：FS-DSLR200\n   - 价格：599.99美元\n   - 主要特点：24.2MP传感器、1080p视频、3英寸LCD、可更换镜头\n   - 描述：使用这款多功能的单反相机，捕捉惊艳的照片和视频。\n   - 保修：1年\n   - 评分：4.7\n\n关于电视：\n我们有以下电视可供选择：\n1. **CineView 4K TV**:\n   - 品牌：CineView\n   - 型号：CV-4K55\n   - 价格：599.99美元\n   - 主要特点：55英寸显示屏、4K分辨率、HDR、智能电视\n   - 描述：一款色彩鲜艳、智能功能丰富的惊艳4K电视。\n   - 保修：2年\n   - 评分：4.8\n\n2. **CineView 8K TV**:\n   - 品牌：CineView\n   - 型号：CV-8K65\n   - 价格：2999.99美元\n   - 主要特点：65英寸显示屏、8K分辨率、HDR、智能电视\n   - 描述：通过这款惊艳的8K电视，体验未来。\n'

## 三、使用 GPT 评估回答是否正确

我们希望您能从中学到一个设计模式，即当您可以指定一个评估 LLM 输出的标准列表时，您实际上可以使用另一个 API 调用来评估您的第一个 LLM 输出。

In [5]:
# 问题、上下文
product_info = []
for k, v in tools.PRODUCTS_BY_CATEGORY.items():
    category = [f"{k}类别："]
    category.extend(v)
    product_info.append("\n".join(category))
cust_prod_info = {
    'customer_msg': customer_msg,
    'context': "\n\n".join(product_info)
}

In [6]:
def eval_with_rubric(test_set, assistant_answer):
    """
    使用 GPT API 评估生成的回答

    参数：
    test_set: 测试集
    assistant_answer: 助手的回复
    """
    
    cust_msg = test_set['customer_msg']
    context = test_set['context']
    completion = assistant_answer
    
    # 人设
    system_message = """\
    你是一位助理，通过查看客户服务代理使用的上下文来评估客户服务代理回答用户问题的情况。
    """

    # 具体指令
    user_message = f"""\
    你正在根据代理使用的上下文评估对问题的提交答案。以下是数据：
    [开始]
    ************
    [用户问题]: {cust_msg}
    ************
    [使用的上下文]: {context}
    ************
    [客户代理的回答]: {completion}
    ************
    [结束]

    请将提交的答案的事实内容与上下文进行比较，忽略样式、语法或标点符号上的差异。
    回答以下问题：
    助手的回应是否只基于所提供的上下文？（是或否）
    回答中是否包含上下文中未提供的信息？（是或否）
    回应与上下文之间是否存在任何不一致之处？（是或否）
    计算用户提出了多少个问题。（输出一个数字）
    对于用户提出的每个问题，是否有相应的回答？
    问题1：（是或否）
    问题2：（是或否）
    ...
    问题N：（是或否）
    在提出的问题数量中，有多少个问题在回答中得到了回应？（输出一个数字）
"""

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = OpenAiChat().get_completion_from_messages(messages)
    return response

In [7]:
evaluation_output = eval_with_rubric(cust_prod_info, _response)
print(evaluation_output)

- 助手的回应是否只基于所提供的上下文？是
- 回答中是否包含上下文中未提供的信息？否
- 回应与上下文之间是否存在任何不一致之处？否
- 计算用户提出了多少个问题。2
- 对于用户提出的每个问题，是否有相应的回答？
  - 问题1：是
  - 问题2：是
- 在提出的问题数量中，有多少个问题在回答中得到了回应？2


## 四、给出一个标准回答，要求其评估生成回答与标准回答的差距

在经典的自然语言处理技术中，有一些传统的度量标准用于衡量 LLM 输出与人类专家编写的输出的相似度。例如，BLUE 分数可用于衡量两段文本的相似程度。

实际上有一种更好的方法，即使用 Prompt。您可以指定 Prompt，使用 Prompt 来比较由 LLM 自动生成的客户服务代理响应与人工理想响应的匹配程度。

In [11]:
'''基于中文Prompt的验证集'''
test_set_ideal = {
    'customer_msg': """\
告诉我有关 the smartx pro phone 和 the fotosnap camera, the dslr one 的信息。\n另外，你们这有什么 TVs ？""",
    'ideal_answer':"""\
SmartX ProPhone是一款功能强大的智能手机，具有6.1英寸的显示屏，128GB的存储空间，12MP双摄像头和5G网络。它的价格为899.99美元，保修期为1年。
FotoSnap DSLR相机是一款功能强大的相机，具有24.2MP传感器，1080p视频，3英寸LCD屏幕和可更换镜头。它的价格为599.99美元，保修期为1年。
我们有多种电视可供选择，包括CineView 4K电视，CineView 8K电视和CineView OLED电视。CineView 4K电视具有55英寸的显示屏，4K分辨率，HDR和智能电视功能，价格为599.99美元，保修期为2年。CineView 8K电视具有65英寸的显示屏，8K分辨率，HDR和智能电视功能，价格为2999.99美元，保修期为2年。CineView OLED电视具有55英寸的显示屏，4K分辨率，HDR和智能电视功能，价格为1499.99美元，保修期为2年
    """
}

In [12]:
def eval_vs_ideal(test_set, assistant_answer):
    """
    评估回复是否与理想答案匹配

    参数：
    test_set: 测试集
    assistant_answer: 助手的回复
    """
    cust_msg = test_set['customer_msg']
    ideal = test_set['ideal_answer']
    completion = assistant_answer
    
    system_message = """\
    您是一位助理，通过将客户服务代理的回答与理想（专家）回答进行比较，评估客户服务代理对用户问题的回答质量。
    请输出一个单独的字母（A 、B、C、D、E），不要包含其他内容。 
    """

    user_message = f"""\
    您正在比较一个给定问题的提交答案和专家答案。数据如下:
    [开始]
    ************
    [问题]: {cust_msg}
    ************
    [专家答案]: {ideal}
    ************
    [提交答案]: {completion}
    ************
    [结束]

    比较提交答案的事实内容与专家答案。忽略样式、语法或标点符号上的差异。
    提交的答案可能是专家答案的子集、超集，或者与之冲突。确定适用的情况，并通过选择以下选项之一回答问题：
    （A）提交的答案是专家答案的子集，并且与之完全一致。
    （B）提交的答案是专家答案的超集，并且与之完全一致。
    （C）提交的答案包含与专家答案完全相同的细节。
    （D）提交的答案与专家答案存在分歧。
    （E）答案存在差异，但从事实的角度来看这些差异并不重要。
    选项：ABCDE
"""

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = OpenAiChat().get_completion_from_messages(messages)
    return response

这个评分标准来自于 OpenAI 开源评估框架，这是一个非常棒的框架，其中包含了许多评估方法，既有 OpenAI 开发人员的贡献，也有更广泛的开源社区的贡献。

在这个评分标准中，我们要求 LLM 针对提交答案与专家答案进行信息内容的比较，并忽略其风格、语法和标点符号等方面的差异，但关键是我们要求它进行比较，并输出从A到E的分数，具体取决于提交的答案是否是专家答案的子集、超集或完全一致，这可能意味着它虚构或编造了一些额外的事实。

LLM 将选择其中最合适的描述。


In [13]:
eval_vs_ideal(test_set_ideal, _response)
# 对于该生成回答，GPT 判断生成内容是标准回答的一个子集

'    选项：C'

In [14]:
assistant_answer_2 = "life is like a box of chocolates"

In [15]:
eval_vs_ideal(test_set_ideal, assistant_answer_2)
# 对于明显异常答案，GPT 判断为不一致

'    选项：D'

希望您从本章中学到两个设计模式。

1. 即使没有专家提供的理想答案，只要能制定一个评估标准，就可以使用一个 LLM 来评估另一个 LLM 的输出。

2. 如果您可以提供一个专家提供的理想答案，那么可以帮助您的 LLM 更好地比较特定助手输出是否与专家提供的理想答案相似。

希望这可以帮助您评估 LLM 系统的输出，以便在开发期间持续监测系统的性能，并使用这些工具不断评估和改进系统的性能。